In [ ]:
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']

import re
from pathlib import Path
import IPython.display as ipd                               
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


In [ ]:
""" Konfiguration: Plots """
sns.set_style("whitegrid")


In [ ]:
save_pfad = 'ergebnis/gesamt/'

"""Konfiguration: Daten einlesen"""

standort_listen = {
    'A': 'daten/A.csv',
    'B': 'daten/B.csv',
    'C': 'daten/C.csv',
    'D': 'daten/D.csv',
    'E': 'daten/E.csv',
    'F': 'daten/F.csv'
}

# Sortierreihenfolge bei Vergleich
# d.h.: Originale werden immer aus dem Bestand genommen
# der vor einem anderen kommt
standorte_nach_prioritaet = [
    # Standorte
    'A',
    'F',
    'E',
    # Stud.
    'B',
    'C',
    'D'
]



interpretiere_als_leer = [
    '0\'00000"',
    '',
    'UNBEKANNT',
    'Unbekannt',
    '-',
    'o. A',
    'o.A.',
    # BNR: Platzhalter-Werte
    '[]',
    '[-]', 
    '[unbekannt]',
    '[PROMO]',
    '[- PROMO]',
    '[[ PROMO]]',
    '[ PROMO]',
    '[Promo]',
    '[Promo-CD]',
    '[o. A.]',
    '[ohne Nummer]',
    '[ohne Nr.]',
    '[o. Nr.]'
    # RHTI: Leere Werte
    ' "',
    ' " ',
    # MIT: Leere Werte
    'Diverse',
    'Nicht genannt',
    'nicht genannt'
]

"""
Sonstige, überdurchschnittlich häufige Werte, ggf. auch als leer interpretieren?
    # RHTI: Allweltstitel 
    'Live',
    'Streichquartette',
    'Lieder',
    'Kammermusik',
    'Live USA',
    'Greatest hits', 
    'Greatest Hits',
    'Die großen Erfolge',
    # MIT: Gruppenbezeichnungen
    'Diverse',
    'Ensemble',
    'Orchester',
    'Original Cast'
"""


for wert in list(interpretiere_als_leer):
    interpretiere_als_leer.append(wert.lower())
    interpretiere_als_leer.append(wert.upper())

mapping_spalte_dtype = {
         'BEST': 'string',
         'ANR': 'string',
         'EAN': 'string',
         'PEAN': 'string',
         'LC': 'string',
         'LN': 'string',
         'BNR':'string',
         'TTS': 'string',
         'MIT': 'string',
         'MIT_TYP': 'string',
         'DAUER': 'string',
         'ABS_DAUER': 'string',
         'T-ISRC': 'string',
         'T-RHTI': 'string',
         'RHTI': 'string',
         'TRÄGER': 'Int64',
         'SEITE': 'Int64',
         'TAKE': 'Int64'
}



In [ ]:
"""Daten einlesen und aufbereiten"""

standort_dataframes = dict()
for standort in standort_listen:
    standort_liste = standort_listen[standort]
    standort_dataframes[standort] = pd.read_csv(standort_listen[standort], sep=';',
                                                index_col=1,
                                                na_values=interpretiere_als_leer,
                                                usecols=lambda x: 'l_' not in x and 'unnamed' not in x.lower(),
                                                dtype=mapping_spalte_dtype,
                                                encoding='latin1')
    
    standort_dataframes[standort].index = standort_dataframes[standort].index.astype(str)


kb = pd.concat([standort_dataframes[standort] for standort in standort_dataframes])
kb['BEST'] = kb['BEST'].astype('category')
kb['BEST'].cat.set_categories(standorte_nach_prioritaet, inplace=True)

kb = kb[kb['TTS'].str.contains('Compact') == True]

kb['EAN/PEAN'] = kb['EAN']
del kb['EAN']
kb['EAN/PEAN'].fillna(kb['PEAN'], inplace=True)
del kb['PEAN']
kb['DAUER'].fillna(kb['ABS_DAUER'], inplace=True)
del kb['ABS_DAUER']
kb['LC'].fillna(kb['LN'], inplace=True)
del kb['LN']


In [ ]:
"""def vereinheitliche(duplicates):
    if len(duplicates) > 1:
        cols = ['MIT', 
                'RHTI', 
                'T-RHTI', 
                'DAUER', 
                'EAN/PEAN', 
                'T-ISRC', 
                'LC', 
                'BNR', 
                'TRÄGER',
                'SEITE',
                'TAKE']
        assign = {col:duplicates.loc[~pd.isnull(duplicates[col])].head(1)[col] for col in cols}
        duplicates = duplicates.assign(**assign)
    return duplicates

kb = kb.groupby(['ANR']).apply(vereinheitliche)"""
kb = kb[~kb.index.duplicated(keep='first')]
del kb['MIT_TYP']
del kb['TTS']
assert kb.index.is_unique == True

In [ ]:
kb.columns

In [ ]:
for standort in standort_listen:
    cols = kb.columns
    mit_wert = [ kb.loc[kb['BEST'] == standort, col].count() for col in cols]
    ohne_wert = [ kb.loc[kb['BEST'] == standort, col].isnull().sum() for col in cols]
    compare_cols = pd.DataFrame({'fehlt': ohne_wert, 'vorhanden': mit_wert}, index=cols)
    ax = compare_cols.plot(kind='barh', stacked=True, title=f'{standort}: Fehlende Daten')
    ax.set(xlabel='Einheiten', ylabel='Spalten')
    plt.title(f'{standort}: Fehlende Werte')
    plt.savefig(f'{save_pfad}/img/{standort}_fehlende_werte.svg')
    plt.show()
    


In [ ]:
kb.columns

In [ ]:
""" Konfiguration: Vergleiche """


ERLAUBTE_BEDINGUNGEN = [
    'schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand',
    'traeger_seite_take_fehlen',
    'bnr_mindestlaenge',
    'bnr_fehlt'
]


vergleiche = dict()
def add_vergleich(name: str,
                  nutze_spalten: list,
                  **bedingungen) -> None:

    #nutze_spalten.sort()
    
    if name in vergleiche:
        raise KeyError(f'Name ({name}) bereits in Vergleich')
    
    if nutze_spalten in vergleiche.values():
        raise ValueError(f'Vergleich ({nutze_spalten}) bereits vorhanden!')
        
    vergleiche[name] = dict()
    
    vergleiche[name] = dict()
    vergleiche[name]['nutze_spalten'] = nutze_spalten
    
    for spalte in nutze_spalten:
        if spalte not in kb.columns:
            raise ValueError(f'Spalte ({spalte}) ist nicht in den Daten.')
    if bedingungen is not None:
        for bedingung in bedingungen:
            if bedingung not in ERLAUBTE_BEDINGUNGEN:
                raise ValueError(f'Bedingung ({bedingung}) ist unbekannt.')
        vergleiche[name].update(bedingungen)
        
    


# Vergleiche
add_vergleich('alles',
              ['TRÄGER', 'SEITE', 'TAKE', 'EAN/PEAN', 'LC', 'BNR', 'MIT', 'RHTI', 'T-ISRC', 'T-RHTI'],
              schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand=True)

add_vergleich('LC, BNR, T-ISRC',
              ['LC', 'BNR', 'T-ISRC'],
             )

add_vergleich('EAN, T-ISRC',
              ['EAN/PEAN', 'T-ISRC'],
              schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand=True)

add_vergleich('RHTI, T-ISRC',
              ['RHTI', 'T-ISRC'],
              schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand=True)


add_vergleich('TST, EAN, LC, BNR, MIT, RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'EAN/PEAN', 'LC', 'BNR', 'MIT', 'RHTI'])

add_vergleich('TST, EAN, RHTI, T-RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'EAN/PEAN', 'T-RHTI', 'RHTI'])

add_vergleich('TST, EAN, LC, BNR, RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'EAN/PEAN', 'LC', 'BNR', 'RHTI'])

add_vergleich('TST, EAN, RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'EAN/PEAN', 'RHTI'])


add_vergleich('EAN, LC, BNR, RHTI',
              ['EAN/PEAN', 'LC', 'BNR', 'RHTI'],
              traeger_seite_take_fehlen=True)

add_vergleich('EAN, LC, BNR',
              ['EAN/PEAN', 'LC', 'BNR'],
              traeger_seite_take_fehlen=True)


add_vergleich('TST, LC, BNR, T-RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'LC', 'BNR', 'T-RHTI'],
             schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand=True)

add_vergleich('LC, BNR, T-RHTI',
              ['LC', 'BNR', 'T-RHTI'],
             traeger_seite_take_fehlen=True)

add_vergleich('LC, BNR, RHTI',
              ['LC', 'BNR', 'RHTI'],
              traeger_seite_take_fehlen=True)

add_vergleich('TST, LC, BNR, RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'LC', 'BNR', 'RHTI'],
              bnr_mindestlaenge=True)

add_vergleich('TST, MIT, RHTI',
              ['TRÄGER', 'SEITE', 'TAKE', 'MIT', 'RHTI'])

add_vergleich('RHTI, T-RHTI' ,
              ['RHTI', 'T-RHTI'],
              traeger_seite_take_fehlen=True,
              bnr_fehlt=True)


add_vergleich('LC, MIT, RHTI',
              ['LC', 'MIT', 'RHTI', 'DAUER'],
              traeger_seite_take_fehlen=True,
              bnr_fehlt=True)

In [ ]:


""" Vergleichen """
print('Bereite Vergleich vor')

# Konfiguration: Vergleich

PATTERN_NON_ALPHANUMERIC = re.compile(r'\W+')
MINIMALER_ABSTAND_DUBLETTE_ORIGINAL = 80
abstand_erfuellt_spalten_name = f'Abstand >= {MINIMALER_ABSTAND_DUBLETTE_ORIGINAL}?'

kb['hat Dubletten?'] = False
kb['ist Dublette?'] = False
kb[abstand_erfuellt_spalten_name] = False
kb['Original ANR'] = np.nan
kb['Original BEST'] = np.nan
kb['Original RHTI'] = np.nan
kb['Fundmethode'] = np.nan
kb['Abstand'] = np.nan
kb['Abstand'] = kb['Abstand'].astype('Int64')
kb['t_ANR'] = kb.index

gefunden = dict()
gefunden['total'] = dict()
ignoriert = 'ignorierte Dubletten (mit weniger Kriterien anderes Original gefunden, zur Sicherheit verworfen (Ergebnisse ohne Abstandsprüfung!)'
gefunden[ignoriert] = 0
gefunden['total']['insgesamt'] = 0
gefunden['total'][abstand_erfuellt_spalten_name] = 0
gefunden['total']['als Dubletten markiert'] = 0

print('Normalisiere Daten')
# Alle Spalten mit Strings normalisieren
normalisiert_kb = kb.copy()
string_spalten = normalisiert_kb.select_dtypes(include=['string'])
normalisiert_kb.loc[:, string_spalten.columns] = string_spalten.apply(lambda x: x.str.lower().str.replace(PATTERN_NON_ALPHANUMERIC, ''))
normalisiert_kb.loc[:, string_spalten.columns].replace('', np.nan, inplace=True)


for reihenfolge, methode in enumerate(vergleiche, 1):
    fundmethode = f'{reihenfolge}: {methode}'
    nutze_spalten = vergleiche[methode]['nutze_spalten']

    print(f'\nVergleiche: {fundmethode} ({reihenfolge}/{len(vergleiche)})')
    print(f'Spalten: {nutze_spalten}')
    print(f"Bedingungen: {[vgl for vgl in vergleiche[methode].keys() if 'nutze_spalten' not in vgl]}")
    
    
    normalisiert = False
    if vergleiche[methode].get('normalisiere', True):
        normalisiert = True
        vgl = normalisiert_kb[(kb['ist Dublette?'] == False)].dropna(subset=nutze_spalten).copy()
    else:
        vgl = kb[(kb['ist Dublette?'] == False)].dropna(subset=nutze_spalten).copy()
        
    if vergleiche[methode].get('bnr_mindestlaenge', False):
        bnr_mindestlaenge = 4
        if normalisiert:
            # '[]' abziehen
            bnr_mindestlaenge -= 2
        vgl.drop( (vgl[ ~(vgl['BNR'].str.len() >= bnr_mindestlaenge) ] ).index, inplace=True)
    
    if vergleiche[methode].get('traeger_seite_take_fehlen', False):
        tst_spalten = ['TRÄGER', 'SEITE', 'TAKE']
        vgl.drop( (vgl[ ~(vgl[tst_spalten].isnull().all('columns')) ] ).index, inplace=True)
    
    
    if vergleiche[methode].get('bnr_fehlt', False):
        vgl.drop( (vgl[ ~(vgl['BNR'].isnull()) ] ).index, inplace=True)
    
    vgl.dropna(subset=nutze_spalten, inplace=True)
    vgl.sort_values(['BEST'], inplace=True)
    
    dubletten = vgl.duplicated(subset=nutze_spalten)
    originale = vgl.groupby(nutze_spalten)['t_ANR'].transform('first').values
    
    vgl['ist Dublette?'] = dubletten
    vgl['Original ANR'] = originale
    
    if len(vgl[vgl['ist Dublette?'] == True]) > 0:
        # Schon als Originale registrierte Einheiten nicht als Dubletten zählen
        gefunden[ignoriert] += vgl.loc[(vgl['ist Dublette?'] == True) & (kb['hat Dubletten?'] == True), 'ist Dublette?'].sum() 
        vgl.loc[ (vgl['ist Dublette?'] == True) & (kb['hat Dubletten?'] == True), 'ist Dublette?'] = False


        vgl['Original ANR'] = vgl['Original ANR']
        vgl['Original BEST'] = kb.loc[vgl['Original ANR'], 'BEST'].values
        vgl['Original RHTI'] = kb.loc[vgl['Original ANR'], 'RHTI'].values

        vgl.drop( vgl[(vgl['ist Dublette?'] == False)].index, inplace=True)
        
        vgl['eigene ANR (numerisch)'] = pd.to_numeric(vgl.index.str.replace(r'[^\d]', '', regex=True))
        vgl['originale ANR (numerisch)'] = pd.to_numeric(vgl['Original ANR'].str.replace(r'[^\d]', '', regex=True))
        vgl['Abstand'] = abs(vgl['eigene ANR (numerisch)'] - vgl['originale ANR (numerisch)'])

        # Ergebnisse übertragen
        if vergleiche[methode].get('schliesse_dubletten_auch_ein_wenn_nicht_mindestabstand', False):
             kb.loc[vgl.index, 'ist Dublette?'] = True
        else:
            kb.loc[vgl.index, 'ist Dublette?'] = vgl['Abstand'] >= MINIMALER_ABSTAND_DUBLETTE_ORIGINAL


        kb.loc[vgl.index, 'Original ANR'] = vgl['Original ANR']
        kb.loc[vgl.index, 'Original BEST'] = vgl['Original BEST']
        kb.loc[vgl.index, 'Original RHTI'] = vgl['Original RHTI']
        kb.loc[kb.index.isin(vgl['Original ANR']), 'hat Dubletten?'] = True
        kb.loc[vgl.index, 'Fundmethode'] = fundmethode
        kb.loc[vgl.index, 'Abstand'] = vgl['Abstand']
        kb.loc[vgl.index, abstand_erfuellt_spalten_name] =  vgl['Abstand'] >= MINIMALER_ABSTAND_DUBLETTE_ORIGINAL
    
    
    gefunden_insgesamt = 0
    gefunden_markiert = 0
    gefunden_abzgl_zu_nah = 0
    if len(vgl[vgl['ist Dublette?'] == True]) > 0:
        gefunden_insgesamt = len(vgl['ist Dublette?'])
        gefunden_markiert = (kb.loc[vgl.index, 'ist Dublette?']).sum()
        gefunden_abzgl_zu_nah = len(vgl[(vgl['ist Dublette?'] == True) & (vgl['Abstand'] >= MINIMALER_ABSTAND_DUBLETTE_ORIGINAL)])
    
    print(f'Gefunden (insgesamt): {gefunden_insgesamt}')
    print(f'Gefunden (ohne zu nahe ANR): {gefunden_abzgl_zu_nah}')
    print(f'Gefunden (als Dubletten markiert): {gefunden_markiert}\n')
    
    gefunden[fundmethode] = dict()
    gefunden[fundmethode]['insgesamt'] = gefunden_insgesamt
    gefunden[fundmethode]['als Dubletten markiert'] = gefunden_markiert
    gefunden[fundmethode][abstand_erfuellt_spalten_name] = gefunden_abzgl_zu_nah
    gefunden['total']['insgesamt'] += gefunden[fundmethode]['insgesamt']
    gefunden['total']['als Dubletten markiert'] += gefunden[fundmethode]['als Dubletten markiert']
    gefunden['total'][abstand_erfuellt_spalten_name] += gefunden[fundmethode][abstand_erfuellt_spalten_name]
    
    del vgl 

del kb['t_ANR']

assert len(kb[ (kb['ist Dublette?'] == True) & (kb.index.isin(kb['Original ANR'])) ]) == 0
assert len(kb[ (kb['ist Dublette?'] == True) & (kb['hat Dubletten?'] == True) ]) == 0
           
ipd.display(gefunden['total'])
print('Fertig!')



In [ ]:
r = dict()
prev = None
for standort in standorte_nach_prioritaet:
    if standort in standort_listen:
        r[f'{standort}: dubletten in {standort}'] = len(kb[(kb['BEST'] == standort) & (kb['ist Dublette?'] == True)])
        r[f'{standort}: dubletten in {standort} mit original in {standort}'] = len(kb[(kb['BEST'] == standort) & (kb['ist Dublette?'] == True) & (kb['Original BEST'] == standort)])
        if prev is not None:
            r[f'{standort}: dubletten in {standort} mit original in {prev}'] = len(kb[(kb['BEST'] == standort) & (kb['ist Dublette?'] == True) & (kb['Original BEST'] == prev)])
        prev = standort
    
r

In [ ]:
g = gefunden.copy()

del g['total']
del g[ignoriert]

for m in g:
    i = g[m]['insgesamt']
    a = g[m][abstand_erfuellt_spalten_name]
    print(f'{i},{a}')

print('\n')
for m in g:
    i = g[m]['insgesamt']
    a = g[m][abstand_erfuellt_spalten_name]
    print(f'{m}: {i},{a}')

In [ ]:
x = kb[(kb['Fundmethode'].str.contains('5') == True) & (kb['RHTI'].str.contains('Vol.') == True) & (kb['Fundmethode'].str.contains('15') == False) & (kb['ist Dublette?'] == True)].sample(10)

x.sort_values('Original ANR')

In [ ]:
y = kb[kb.index.isin(x['Original ANR'])].sort_values('Original ANR')

y

In [ ]:
x = kb.loc[kb['ist Dublette?'] == True, 'Fundmethode'].value_counts()
y =  x.index
ax = sns.barplot(x=x, y=y)
ax.set(ylabel='Reihenfolge: Vergleichsmethode', xlabel='Dubletten (abs.)')
plt.title('Fundmethoden nach Fundmenge')
plt.savefig(f'{save_pfad}/img/fundmethoden.svg', bbox_inches='tight')
plt.show()



x = kb.loc[(kb['ist Dublette?'] == True), ['BEST', 'Original BEST']].value_counts().unstack().sort_values('BEST')
ax = x.plot(kind='bar')
plt.title('Wo stehen die Originale der Dubletten?')
ax.legend(title='Standort des Originals')
ax.set(ylabel='Originale', xlabel='Standort der Dublette')
plt.savefig(f'{save_pfad}/img/dubletten_originale.svg', bbox_inches='tight')
plt.show()

    
x = kb.loc[:, ['BEST', 'ist Dublette?']].value_counts()
ax = x.unstack(level=1).plot(kind='bar', stacked=True)
ax.legend(['Original', 'Dublette'])
plt.title('Dubletten nach Standort (bezogen auf die jeweilige Gesamtmenge)')
ax.set(ylabel='Einheiten', xlabel='Standort')
plt.savefig(f'{save_pfad}/img/dubletten_nach_standort.svg', bbox_inches='tight')
plt.show()

for standort in standorte_nach_prioritaet: 
    if standort in standort_listen:
        x = kb.loc[kb['BEST'] == standort, ['BEST', 'ist Dublette?']].value_counts(normalize=True) * 100
        ax = x.unstack().plot(kind='bar', stacked=True)
        ax.legend(['Original', 'Dublette'])
        plt.title(f'{standort}: Dubletten im Bestand (prozentual)')
        ax.set(ylabel='Einheiten (%)', xlabel=None)
        plt.savefig(f'{save_pfad}/img/{standort}_dubletten_nach_standort.svg', bbox_inches='tight')
        plt.show()


In [ ]:
results_file = Path(f'{save_pfad}/ergebnis.xlsx')
if results_file.exists():
    results_file.unlink()

out = kb.copy()
bool_cols = list()
for col, col_type in dict(out.dtypes).items():
    if col_type == 'bool':
        out[col] = out[col].astype('object')
        out.loc[(out[col] == True), col] = 'x'
        out.loc[(out[col] == False), col] = np.nan

        
with pd.ExcelWriter(results_file) as writer:
    for standort in standorte_nach_prioritaet:
        if standort in standort_listen:
            out.to_excel(writer, sheet_name='Alles')
            out.loc[out['BEST'] == standort, :].to_excel(writer, sheet_name=f'{standort} Alles')
            out.loc[out['BEST'] == standort, :].sample(500).to_excel(writer, sheet_name=f'{standort} Alles (500)')
            out.loc[(out['BEST'] == standort) & (out['ist Dublette?'] == 'x'), :].to_excel(writer, sheet_name=f'{standort} Dubletten')

del out
        